# Code Execution with AI Agents in .NET


## Use Cases for AI-Powered Code Execution

Automated Code Generation
  - Boilerplate code, unit tests, or SQL queries, 
  - Reduce the time spent on repetitive and mundane coding tasks

Code Validation & Debugging
- Suggest code improvements 
- Run code snippets in a controlled sandbox environment to identify and fix potential bugs


DevOps & Scripting
- Execute and manage scripts for automation, deployment, or monitoring tasks

Interactive Coding Assistants
- Real-time assistants to explain, modify, or refactor code, helping developers to understand unfamiliar codebases and improve code quality faster

## Configuration

Like in the previous exercise we need to setup LLM and configure keys and endpoints

In [1]:
var Endpoint = "<REPLACE_WITH_AZURE_OPENAI_ENDPOINT>";
var Key = "<REPLACE_WITH_AZURE_OPENAI_API_KEY>";
var ChatModelId = "gpt-4o"; // use any model you deployed

## Creating code execution agent

In [2]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3"
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"
#r "nuget: Microsoft.CodeAnalysis.CSharp.Scripting, 4.13.0"

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 Microsoft.CodeAnalysis.CSharp.Scripting, 4.13.0 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3

In [5]:
using System.ComponentModel;
using System.ClientModel;
using Microsoft.CodeAnalysis.CSharp.Scripting;
using Microsoft.CodeAnalysis.Scripting;
using Microsoft.Extensions.AI;
using Azure.AI.OpenAI;

In [6]:
var client = new AzureOpenAIClient(new Uri(Endpoint), new ApiKeyCredential(Key))
        .AsChatClient(ChatModelId)
        .AsBuilder()
        .UseFunctionInvocation() // allows to call registered functions
        .Build();

### Creating Tools

Now let's create a tool for our client to execute code and register that tool

In [7]:
[Description("Executes a C# expression and returns the result.")]
async Task<string> ExecuteCSharpCode(string code)
{
    try
    {
        Console.WriteLine("executing ExecuteCSharpCode method");
        var result = await CSharpScript.EvaluateAsync<object>(code, ScriptOptions.Default);
        return result?.ToString() ?? "null";
    }
    catch (Exception ex)
    {
        return $"Error: {ex.Message}";
    }
}

In [8]:
var chatOptions = new ChatOptions
{
    Tools =
    [
        AIFunctionFactory.Create(ExecuteCSharpCode)
    ]
};

Now let's ask our agent to calculate something for us

In [ ]:
var chatMessage = "Evaluate: 2 + 2 * 3";
await foreach (var message in client.GetStreamingResponseAsync(chatMessage, chatOptions))
{
    Console.Write(message);
}

We created a tool 'ExecuteCSharpCode' which we provided to our agent.  
With that tool our agent is able to execute code and give us the result. 
This is very powerful technique because now our agent is able to generate code, analyze and execute it, but at the same time it's extremely dangerous. 

>In  our previous example we provided our agent tools that were preconfigured and determined. Our agent could just get weather related information. Right now it can potentially generate a harmful script and run it in our environment  

## Security Concerns


**Security Risks** in Code Execution 
- Risk of Arbitrary Code Execution (RCE) -> AI might generate and run harmful commands.
- Unauthorized Access -> AI might access files, networks, or system processes.
- Infinite Loops & Performance Issues -> AI-generated code could lead to resource exhaustion.


**Risk Mitigations** 
- Sandboxing -> Execute code in an isolated environment (e.g., Roslyn Scripting).
- Resource Limits -> Set max CPU, memory, and execution time.
- API Whitelisting -> Only allow safe pre-approved operations.
- Input Validation -> Analyze generated code before running it.


Let's update our method to make it more safe
- restrict namespaces
- restrict assemblies
- limit execution time

In [10]:
using System.Threading;

private static async Task<object> ExecuteSandboxedCodeAsync(string code)
{
    Console.WriteLine("executing ExecuteCSharpCode method safely");
    // Define sandboxed script options
    var scriptOptions = ScriptOptions.Default
        .WithImports("System") // Restrict imports to only safe namespaces
        .WithReferences(AppDomain.CurrentDomain.GetAssemblies());

    using var cts = new CancellationTokenSource(TimeSpan.FromSeconds(3)); // Limit execution time

    try
    {
        return await CSharpScript.EvaluateAsync<object>(code, scriptOptions, cancellationToken: cts.Token);
    }
    catch (CompilationErrorException ex)
    {
        return $"Compilation Error: {string.Join("\n", ex.Diagnostics)}";
    }
    catch (Exception ex)
    {
        return $"Execution Error: {ex.Message}";
    }
}

Now we can use this method which is much safer than directly calling CSharpScript.EvaluateAsync
>//var result = await CSharpScript.EvaluateAsync<object>(code, ScriptOptions.Default);  
var result = await ExecuteSandboxedCodeAsync(code);  

But please note this is a very powerful tool and you need to be very mindful about where and how you can use it

### Final Thoughts

We learnt how to create code execution AI agent in .NET and learnt about the dangers of this technique